In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import math
import re
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
!pwd

/Users/shijunyi/Desktop/学习资料/U of Melbourne/2019 Sem2/Data Science Project/[6] code/python


In [2]:
pd.options.display.precision = 2
pd.options.display.max_columns = 50
matplotlib.rcParams['figure.figsize'] = [15, 10]
init_notebook_mode(connected=True)

In [3]:
src_countries = ['China', 'India', 'Indonesia', 'Malaysia']

In [5]:
for src in src_countries:
    #read data
    factors_df = pd.read_csv('data/grouped_'+src+'.csv')
    
    #calculate correalation and p-value for each destination:
    corr_pvalues={}
    dest_countries = ['United States of America', 'United Kingdom', 'Australia', 'Canada', 'New Zealand']
    
    #filter record for each destination:
    for dest in dest_countries:
        dest_factors = factors_df[factors_df['destination_country']== dest]
        dest_factors = dest_factors.drop(['source_country', 'destination_country', 'dst_distance'],axis=1)
        #shift predictors by 1 and 2 years:
        cols = dest_factors.columns
        dest_factors = dest_factors.join(dest_factors[cols].shift(1), rsuffix='_1y_shift')
        dest_factors = dest_factors.join(dest_factors[cols].shift(2), rsuffix='_2y_shift')
        #fill NaN values:
        dest_factors['year_1y_shift'].iloc[0]=2015
        for col in dest_factors.columns:
            if math.isnan(dest_factors[col].iloc[1]):
                dest_factors[col].iloc[1] = dest_factors[col].iloc[2]
            if math.isnan(dest_factors[col].iloc[0]):
                dest_factors[col].iloc[0] = dest_factors[col].iloc[1]
        #calculate corr and p-valueL
        dic = {}
        for col in dest_factors.columns:
            try:
                c_p_pair = pearsonr(dest_factors[col],dest_factors['dst_students_count'])
            except:
                c_p_pair = (np.nan, np.nan)
            dic[col]=c_p_pair
        corr_pvalues[dest] = dic        
    corr_pvalues_df = pd.DataFrame(corr_pvalues)
    corr_pvalues_df = corr_pvalues_df.drop('dst_students_count')
    
    
    [rows, cols] = corr_pvalues_df.values.shape
    for i in range(rows):
        for j in range(cols):
            if corr_pvalues_df.values[i][j][1] <= 0.05:  #reject H0, the corr is sig
                corr_pvalues_df.values[i][j]  = corr_pvalues_df.values[i][j][0]
            else:
                corr_pvalues_df.values[i][j] = np.nan  
          
        
    trace = go.Heatmap(z=corr_pvalues_df.values,
                       x=corr_pvalues_df.columns.values,
                       y=corr_pvalues_df.index.values,
                       colorscale='spectral',
                       reversescale=True)

    layout = go.Layout(
        title="Correlation Between Predcitors and Student Count from "+src,
        autosize=False,
        width=1000,
        height=1000)

    fig = go.Figure(data=trace, layout=layout)
    iplot(fig)